In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from datetime import datetime
import os
from urllib.parse import urlparse


def setup_driver():
    options = Options()
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


def aceptar_cookies(driver):
    try:
        wait = WebDriverWait(driver, 10)
        boton_aceptar = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#didomi-notice-agree-button")))
        boton_aceptar.click()
        print("🍪 Cookies aceptadas.")
        time.sleep(2)
    except:
        print("⚠️ Cookies ya aceptadas o no detectadas.")


def cerrar_popup(driver):
    try:
        popup_close = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.modal__close"))
        )
        popup_close.click()
        print("❌ Modal cerrado.")
        time.sleep(1)
    except:
        pass


def scrape_distrito(distrito):
    base_url = f"https://www.pisos.com/venta/pisos-madrid_{distrito}/"
    driver = setup_driver()
    driver.get(base_url)
    wait = WebDriverWait(driver, 10)
    aceptar_cookies(driver)

    resultados = []
    urls_vistas = set()
    page_num = 1
    MAX_PAGINAS = 300

    while True:
        print(f"\n🔍 [{distrito}] Procesando página {page_num}...")

        try:
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.ad-preview")))
            anuncios = driver.find_elements(By.CSS_SELECTOR, "div.ad-preview")
            print(f"📦 Anuncios detectados: {len(anuncios)}")
        except:
            print("⚠️ No se detectaron anuncios.")
            break

        nuevos_resultados = 0

        for anuncio in anuncios:
            try:
                link = anuncio.find_element(By.CSS_SELECTOR, "a.ad-preview__title").get_attribute("href").strip()
                url_base = urlparse(link).path
                if url_base in urls_vistas:
                    continue

                titulo = anuncio.find_element(By.CSS_SELECTOR, "a.ad-preview__title").text.strip()
                ubicacion = anuncio.find_element(By.CSS_SELECTOR, "p.ad-preview__subtitle").text.strip()
                precio = anuncio.find_element(By.CSS_SELECTOR, "span.ad-preview__price").text.strip()

                caracteristicas = anuncio.find_elements(By.CSS_SELECTOR, "p.ad-preview__char.p-sm")
                habitaciones = caracteristicas[0].text.strip() if len(caracteristicas) > 0 else ""
                baños = caracteristicas[1].text.strip() if len(caracteristicas) > 1 else ""
                metros = caracteristicas[2].text.strip() if len(caracteristicas) > 2 else ""

                resultados.append({
                    "timestamp": datetime.now().isoformat(),
                    "titulo": titulo,
                    "precio": precio,
                    "ubicacion": ubicacion,
                    "habitaciones": habitaciones,
                    "baños": baños,
                    "metros": metros,
                    "link": link
                })

                urls_vistas.add(url_base)
                nuevos_resultados += 1

            except Exception as e:
                print(f"⚠️ Error extrayendo un anuncio: {e}")
                continue

        print(f"✅ Nuevos resultados esta página: {nuevos_resultados}")
        print(f"📊 Total acumulado en {distrito}: {len(resultados)}")

        if page_num >= MAX_PAGINAS:
            print("🚫 Límite de páginas alcanzado para este distrito.")
            break

        try:
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div.pagination__next a"))
            )
            print("➡️ Clic en 'Siguiente'")
            driver.execute_script("arguments[0].click();", next_button)
            page_num += 1
            time.sleep(5)
            cerrar_popup(driver)
        except:
            print("🚫 No se encontró el botón 'Siguiente'. Fin del scraping para este distrito.")
            break

    driver.quit()
    return resultados


def save_to_json(data, filename="data/venta_madrid_distritos.json"):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"\n💾 Archivo guardado: {filename} ({len(data)} registros)")


if __name__ == "__main__":
    distritos = [
        "arganzuela", "barajas", "carabanchel", "centro", "chamartin", "chamberi",
        "ciudad_lineal", "fuencarral_el_pardo", "hortaleza", "latina",
        "moncloa_aravaca", "moratalaz", "puente_vallecas", "retiro",
        "salamanca", "san_blas", "tetuan", "usera", "vicalvaro", "villa_vallecas", "villaverde"
    ]

    todos_resultados = []
    links_totales = set()

    for distrito in distritos:
        resultados_distrito = scrape_distrito(distrito)
        for r in resultados_distrito:
            base_url = urlparse(r["link"]).path
            if base_url not in links_totales:
                todos_resultados.append(r)
                links_totales.add(base_url)

    print(f"\n📊 Total final de inmuebles únicos: {len(todos_resultados)}")
    save_to_json(todos_resultados)

🍪 Cookies aceptadas.

🔍 [arganzuela] Procesando página 1...
⚠️ No se detectaron anuncios.
🍪 Cookies aceptadas.

🔍 [barajas] Procesando página 1...
📦 Anuncios detectados: 38
✅ Nuevos resultados esta página: 38
📊 Total acumulado en barajas: 38
🚫 No se encontró el botón 'Siguiente'. Fin del scraping para este distrito.
🍪 Cookies aceptadas.

🔍 [carabanchel] Procesando página 1...
📦 Anuncios detectados: 33
✅ Nuevos resultados esta página: 33
📊 Total acumulado en carabanchel: 33
➡️ Clic en 'Siguiente'
❌ Modal cerrado.

🔍 [carabanchel] Procesando página 2...
📦 Anuncios detectados: 30
✅ Nuevos resultados esta página: 27
📊 Total acumulado en carabanchel: 60
➡️ Clic en 'Siguiente'

🔍 [carabanchel] Procesando página 3...
📦 Anuncios detectados: 30
✅ Nuevos resultados esta página: 30
📊 Total acumulado en carabanchel: 90
➡️ Clic en 'Siguiente'

🔍 [carabanchel] Procesando página 4...
📦 Anuncios detectados: 30
✅ Nuevos resultados esta página: 30
📊 Total acumulado en carabanchel: 120
➡️ Clic en 'Siguie